In [13]:
#importing the necessay dependancies
import random
random.seed(0)
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(0)
from tensorflow.keras import models, layers




In [14]:
# Image parameters
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 10  # Increased the number of epochs for better training



In [ ]:

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    r"D:\Projects\Plant Disease\Tomato",  # Use raw string to avoid escape characters
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)


In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch.shape)
    print(image_batch[0].shape)

In [19]:
class_names = dataset.class_names
n_classes = len(class_names)


In [ ]:
# Plotting the dataset images
plt.figure(figsize=(15, 15))
for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        axis = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

plt.show()

In [21]:
# Splitting the dataset
train_size = 0.8
val_size = 0.1
test_size = 0.1

In [22]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size= 10000):
    df_size = len(dataset)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed =12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [23]:
dataset_size = dataset.cardinality().numpy()
dataset = dataset.shuffle(buffer_size=dataset_size)

In [24]:
train_ds = dataset.take(int(dataset_size * train_size))
test_ds = dataset.skip(int(dataset_size * train_size))
val_ds = test_ds.take(int(dataset_size * val_size))
test_ds = test_ds.skip(int(dataset_size * val_size))

In [25]:
# Optimizing dataset with caching, shuffling, and prefetching
train_ds = train_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [26]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    tf.keras.layers.Rescaling(1.0/255)
])

In [27]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomZoom(0.2),        # Random zoom
    layers.RandomTranslation(0.2, 0.2),  # Random translation
    layers.RandomRotation(0.2),     # Optionally keep rotation
    layers.RandomContrast(0.2)      # Random contrast adjustment
])

In [ ]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

# Building the model
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),  # Add another Conv layer
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'), # Add a deeper Conv layer
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'), # Add a deeper Conv layer
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'), # Add a deeper Conv layer
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),  # Increase units in dense layer
    layers.Dense(n_classes, activation='softmax')
])


model.summary()



In [29]:
# Model compilation
model.compile(
    optimizer = "adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [30]:
#  Training the model
history = model.fit(
    train_ds,
    epochs=10,
    verbose=1,
    validation_data=val_ds,
)

KeyboardInterrupt: 

In [ ]:

# Evaluating the model
scores = model.evaluate(test_ds)
print(f"Test Loss: {scores[0]}, Test Accuracy: {scores[1]}")


In [ ]:
history

In [ ]:
history.params

In [ ]:
history.history.keys()

In [74]:


# Accessing training accuracy, loss, and validation metrics
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:


# Plotting accuracy
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs') # X label for clarity
plt.ylabel('Accuracy') # Y label for clarity
plt.show()


In [ ]:

# Plotting loss
plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()



In [ ]:

# Predicting and displaying the first image from the test set
for image_batch, labels_batch in test_ds.take(1):
    first_image = image_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    plt.imshow(first_image)
    plt.axis('off')  # Hide axes
    plt.title('Actual label: ' + class_names[first_label])
    plt.show()

    batch_prediction = model.predict(image_batch)
    print('Predicted label:', class_names[np.argmax(batch_prediction[0])])


In [92]:
# Prediction function
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    image_array = tf.expand_dims(img_array, 0)  # create batch
    predictions = model.predict(image_array)

    predicted_class = np.argmax(predictions[0])
    confidence = round(100 * np.max(predictions[0]), 2)
    return predicted_class, confidence

In [ ]:
# Display predictions for images in the test dataset
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):  # Display 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i].numpy()]

        plt.title(f"Actual: {actual_class}\nPredicted: {class_names[predicted_class]}\nConfidence: {confidence}%")
        plt.axis("off")

plt.show()

In [ ]:

print("Current working directory:", os.getcwd())
print("Model will be saved to:", model_dir)


In [ ]:
model_version